In [ ]:
import wandb
wandb.login()

In [ ]:
pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch.nn as nn
import h5py
import pickle
import torch
import time
import yaml
import copy
import math
import traceback
import sys
import time
import os
import torch.nn.functional as F
import torchvision
import transformers
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from copy import deepcopy

In [ ]:
import yaml
with open('/home/quanhhh/Documents/RUL_XAI/XAI_env/config.yml', 'r') as f:
    config_wdb = yaml.safe_load(f)

In [ ]:
config_wdb

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, config, x_path, y_path):
        super().__init__()
        self.config = config
        self.x_path = x_path
        self.y_path = y_path
        self.load_dataset()

    def __len__(self):
        return self.data_.shape[0]

    def __getitem__(self, idx):
        input_tensor = torch.from_numpy(self.data_[idx]).float()
        label_tensor = torch.from_numpy(np.array(self.labels[idx])).float()
        return {'input': input_tensor, 'labels': label_tensor}
    def load_dataset(self):
        x_data = {}
        y_data = {}
        x_path_ = os.path.join(self.config['data_dir'], self.x_path)
        y_path_ = os.path.join(self.config['data_dir'], self.y_path)

        with h5py.File(x_path_, 'r') as x_file:
            x_data_key = list(x_file.keys())[0]
            x_data['data'] = np.array(x_file[x_data_key])

        with h5py.File(y_path_, 'r') as y_file:
            y_data_key = list(y_file.keys())[0]
            y_data['label'] = np.array(y_file[y_data_key])

        self.data_ = x_data['data']
        self.labels = y_data['label']
        self.config['data_shape'] = self.data_.shape[1:]

    def getshape(self):
        return self.config['data_shape']


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, input_size,embed_dim, noise_level):
        super(Autoencoder, self).__init__()
        self.embed_dim = embed_dim
        self.input_size, self.hidden_dim, self.noise_level = input_size, embed_dim,noise_level
        self.fc1 = nn.Linear(self.input_size, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, self.input_size)

    def encoder(self,x):
        x = self.fc1(x)
        h1 = F.relu(x)
        return h1

    def mask(self,x):
        corrupted_x = x + self.noise_level + torch.randn_like(x)   # randn_like  Initializes a tensor where all the elements are sampled from a normal distribution.
        return corrupted_x

    def decoder(self, x):
        h2 = self.fc2(x)
        return h2

    def forward (self, x):
        out = self.mask(x) # Adding noise to feed the network
        encoder = self.encoder(out)
        decoder = self.decoder(encoder)
        return decoder
    ## Transformer
    ### Positional encoding
class TransformerAE(nn.Module):
        def __init__(self, input_sizet,noise_level,embed_dim, hidden_size, num_layers, num_heads, dropout):
            super(TransformerAE, self).__init__()

            self.embedding = nn.Linear(input_sizet, hidden_size)
            self.transformer = nn.Transformer(
                d_model=hidden_size,
                nhead=num_heads,
                num_encoder_layers=num_layers,
                num_decoder_layers=num_layers,
                dim_feedforward=hidden_size,
                dropout=dropout
            )
            self.fc = nn.Linear(hidden_size, input_sizet)
            self.autoencoder = Autoencoder(input_size = input_sizet, noise_level = noise_level, embed_dim = embed_dim)

        def forward(self, input_sizet):
            decoder = self.autoencoder(input_sizet)
            embedded = self.embedding(decoder)
            embedded = embedded.permute(1, 0, 2)  # [sequence_len, batch_size, hidden_size]
            encoded = self.transformer(embedded, embedded)  # Self-attention n the input sequence
            encoded = encoded.permute(1, 0, 2)  # [batch_size, sequence_len, hidden_size]
            decoded = self.fc(encoded)
            return decoded


In [ ]:
#Trainer
class ModelTrainer():
    def __init__(self, model, train_data, criterion, optimizer, device, config):
        self.model = model
        self.train_data = train_data
        self.device = device
        self.config = config
        self.train_loss_list = list()
        self.min_loss = float('inf')
        self.best_model = None
        self.best_optimizer = None
        self.optimizer = optimizer
        self.criterion = criterion

    def train_epoch(self, epoch):
        train_loss = 0.0
        self.model.train()
        for i, batch in enumerate(self.train_data):
            x = batch["input"].to(device)
            rul = batch["labels"].to(device)
            rul = torch.reshape(rul[:, 1], (-1, 1))
            x = x.unsqueeze(1)
            out = self.model(x)
            loss = self.criterion(out,rul)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            train_loss += loss


        train_loss = train_loss / len(self.train_data)
        wandb.log({"train loss": train_loss})
        self.train_loss_list.append(train_loss)

        if train_loss < self.min_loss:
            self.min_loss = train_loss
            self.best_model = deepcopy(self.model.state_dict())
            self.best_optimizer = deepcopy(self.optimizer.state_dict())
            self.best_epoch_in_round = epoch
    # Set up the epoch for trainning process
    def train(self):
        try:

            self.model.to(self.device)

            for epoch in range(1, 100):
                self.train_epoch(epoch)
                wandb.log({"epoch": epoch})

                self.config['train_loss_list'] = self.train_loss_list
        except Exception:
            traceback.print_exc()

    def update_config(self):
        return self.config




In [ ]:
sweep_id = wandb.sweep(sweep=config_wdb, project='RUL_Bearing')

In [ ]:
#Train
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def training():
    with wandb.init(config = config_wdb):
        config = wandb.config
        train_data = CustomDataset(config, '/home/quanhhh/Documents/RUL_XAI/XAI_env/data_processed/IMS/x_train.hdf5','/home/quanhhh/Documents/RUL_XAI/XAI_env/data_processed/IMS/y_train.hdf5')
        train_loader = DataLoader(train_data,
                                  batch_size=128,
                                  shuffle=True)
        shape = train_data.getshape()
        model = TransformerAE(input_sizet = shape[0],
            noise_level = 0.01,
            embed_dim = 16,
            hidden_size = 64,
            num_layers = 2,
            num_heads = 4,
            dropout = 0.1)

        optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"], weight_decay=config['weight_decay'])
        criterion = nn.MSELoss()
        trainer = ModelTrainer(model = model,
                               train_data = train_loader,
                               criterion = criterion ,
                               optimizer = optimizer,
                               device = device,
                               config = config)
        trainer.train()

        test_data = CustomDataset(config,'/home/quanhhh/Documents/RUL_XAI/XAI_env/data_processed/IMS/x_test.hdf5','/home/quanhhh/Documents/RUL_XAI/XAI_env/data_processed/IMS/y_test.hdf5')
        test_loader = DataLoader(test_data,
                                  batch_size=128,
                                  shuffle=True)
        model.to(device)
        test_loss = 0.0
        test_loss_list = list()
        pred_list = list()
        truth_list = list()
        with torch.no_grad():
            for i, batch in enumerate(test_loader):
                x = batch["input"].to(device)
                rul = batch["labels"].to(device)
                rul = torch.reshape(rul[:, 1], (-1, 1))
                x = x.unsqueeze(1)
                out = model(x)
                loss = criterion(out,rul)
                test_loss += loss
                test_loss_list.append(loss)
                pred_list.append(out)

        test_loss_avg = test_loss / len(test_loader)

        for x, rul in enumerate(test_loader):
          rul = rul["labels"]
          tensor_values = [value.item() for value in rul.flatten()]
          truth_list.extend(tensor_values)

        #truth_list = [rul["labels"][0].item() for x, rul in enumerate(test_loader)] #####
        #truth_list = [rul.item() for batch in test_loader for rul in batch["labels"]]
        config['truth_list'] = truth_list
        config['pred_list'] = pred_list
        config['test_loss_avg'] = test_loss_avg
        config['test_loss_list_per_id'] = test_loss_list
        wandb.log({"test_loss_avg": test_loss_avg})

        val_data = CustomDataset(config,'/home/quanhhh/Documents/RUL_XAI/XAI_env/data_processed/IMS/x_val.hdf5','/home/quanhhh/Documents/RUL_XAI/XAI_env/data_processed/IMS/y_val.hdf5' )
        val_loader = DataLoader(val_data,
                                 batch_size=128,
                                 shuffle=True)
        model.to(device)
        val_loss = 0.0
        val_loss_list = list()
        with torch.no_grad():
            for i, batch in enumerate(val_loader):
                x = batch["input"].to(device)
                rul = batch["labels"].to(device)
                rul = torch.reshape(rul[:, 1], (-1, 1))
                x = x.unsqueeze(1)
                out = model(x)
                loss = criterion(out,rul)
                val_loss += loss
                val_loss_list.append(loss)
        val_loss_avg = val_loss / len(test_loader)
        config['val_loss_avg'] = test_loss_avg
        config['val_loss_list_per_id'] = test_loss_list
        wandb.log({"val_loss_avg": test_loss_avg})


In [ ]:
!date
wandb.agent(sweep_id, function=training ,count = 200)